<a href="https://colab.research.google.com/github/annemariet/tutorials/blob/master/03_ML_and_dataviz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML4Dataviz / Dataviz4ML, let's practice!

This practical is organized in two parts. **The first is "ML4Dataviz"**: we are going to explore the MNIST dataset using high-dimensional projections. The goal is that you get a feeling of how complex datasets can be visualized through classical linear and non-linear projections.

- You are expected to write mostly the `dataviz` part of the code (ie `matplotlib`, `seaborn`, etc)
- You are expected to play with the parameters of the projection functions and explain how they impact performance & display.

**The second part is "Dataviz4ML"**: We will play with the Fashion-MNIST dataset. A simple feed-forward neural network is implemented. Knowing `keras` is not a prerequisite. The goal in this section is to work you through some examples of how visualization will help you understand & debug your models.

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn import random_projection, decomposition, manifold
import numpy as np
from time import time

import matplotlib.pyplot as plt
from matplotlib import offsetbox

# 1. Visualizing MNIST

We use the `scikit-learn` library to download the data and matplotlib for preliminary basic visualizations.

In [ ]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True)
X = X / 255.
y = y.astype(np.int)

For simplicity we will only work on the first 1000 samples. Not that the usual split uses the first 60k samples for training and the rest for test.

In [ ]:
n_vis_samples = 1000
w, h = 28, 28
X_vis = X[:n_vis_samples]
y_vis = y[:n_vis_samples]

It's a good idea to have a look at the raw data to begin with. In the specific case of images we can use a the matplotlib function `imshow` and choose the colormap we prefer. 

Check the difference between `gray` and `gray_r` colormaps.



In [ ]:
ax = plt.subplot(111)
ax.imshow(X_vis[0,:].reshape(w, h), cmap=plt.cm.gray_r)
ax.set_title(y_vis[0])
ax.set_axis_off();

We're going to plot images often, so it might help to have a function for this. Define a function plot_image using the following prototype:

In [ ]:
def plot_image(img, reshape_size=None, cmap="binary", ax=None, title=None):
  raise NotImplementedError("TODO")
  return ax

Now you can use this function to plot a bunch of images from the dataset in a grid and have a feeling of what MNIST digits look like. you can play around with the parameters such as colormaps, grid size, etc..

You can use the [`plt.subplots`](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplots.html) from matplotlib to draw several axes on the same figure.

In [ ]:
#TODO

## High-dimensional projections to 2D

In this exercise we are going to classical methods to visualize MNIST data in 2D.

The following sample code is adapted from the [scikit-learn documentation](https://scikit-learn.org/stable/auto_examples/manifold/plot_lle_digits.html#sphx-glr-auto-examples-manifold-plot-lle-digits-py).

What is the original dimension of MNIST features?

In [ ]:
#TODO

In [ ]:
def plot_digits_embedding(X2d, y, title=None, remove_ticks=True):
  """
  Plot a 2D points at positions `X2d` using text labels from `y`.
  The data is automatically centered and rescaled to [0,1]
  """
  x_min, x_max = np.min(X2d, 0), np.max(X2d, 0)
  X = (X2d - x_min) / (x_max - x_min)

  plt.figure(figsize=(20,10))
  ax = plt.subplot(111)
  for i in range(X.shape[0]):
    plt.text(X[i, 0], X[i, 1], str(y[i]),
                color=plt.cm.tab10(y[i]),
                fontdict={'weight': 'bold', 'size': 9})

  if remove_ticks:
    plt.xticks([]), plt.yticks([])
  if title is not None:
    plt.title(title)

We start with a random projection. Even though projecting randomly in 2 dimensions is a rather bad idea, when the original dimension is huge, a random projection to a smaller, still high, dimension might give a nice speedup.

In [ ]:
print("Computing random projection")
t0 = time()
rp = random_projection.SparseRandomProjection(n_components=2, random_state=42)
X_projected = rp.fit_transform(X_vis)
plot_digits_embedding(X_projected, y_vis, "Random Projection of the digits (time %.2fs)" %
               (time() - t0))


### PCA

PCA is the go-to method for dimensionality reduction (not only for visualisation). Play around with it to get a feel of how it works.

You can use scikit-learn's PCA. What is it using under the hood?

In [ ]:
print("Computing PCA projection")
t0 = time()
pca = decomposition.PCA(n_components=2, svd_solver="auto")
X_pca = pca.fit_transform(X_vis)
plot_digits_embedding(X_pca, y_vis, 
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0),
               remove_ticks=False)

 Compare the different `svd_solver` efficiency. `%timeit` is a notebook magic that allows you to compute the time taken by a line of code. Which method is best if you use the full dataset, vs only a few?

In [ ]:
#TODO %timeit

Since PCA is a linear decomposition, each component is itself an image. Let's have a look at these "eigenimages". They are stored in the `components_` attribute of the sklearn `PCA` object.

In [ ]:
#TODO

Do you think the projection is good? Can you quantify how good it is? How can you explain this? 

In [ ]:
#TODO

Can non-linear method give us better results here?

### Multidimensional Scaling (MDS)

In [ ]:
print("Computing MDS embedding")
clf = manifold.MDS(n_components=2, n_init=1, max_iter=100)
t0 = time()
X_mds = clf.fit_transform(X[:3000])
print("Done. Stress: %f" % clf.stress_)
plot_digits_embedding(X_mds[:n_vis_samples], y_vis, 
               "MDS embedding of the digits (time %.2fs)" %
               (time() - t0))

### t-SNE

Play with the various parameters and observe how time/cluster change. [Scikit-learn documentation](https://scikit-learn.org/stable/modules/manifold.html#t-sne) is a good starting point if you want to know more.

- perplexity (`perplexity`)
- early exaggeration factor (`early_exaggeration`)
- learning rate (`learning_rate`)
- maximum number of iterations (`n_iter`)
- angle (`angle`, not used in the exact method)

Which parameters have more impact?

In [ ]:
tsne = manifold.TSNE(n_components=2, init='random', random_state=42)
t0 = time()
X_tsne = tsne.fit_transform(X_vis)

plot_digits_embedding(X_tsne, y_vis, 
               "t-SNE embedding of the digits (time %.2fs)" %
               (time() - t0))


### UMAP

UMAP is not yet integrated into scikit-learn but is available as a standalone library that follows the sklearn API. You can have a look at [their doc](https://umap-learn.readthedocs.io/en/latest/parameters.html) to get a better understanding of the parameters.

In [ ]:
#!pip install umap-learn # run this if you run into an "ModuleNotFoundError: No module named 'umap'" error below

In [ ]:
import umap

um = umap.UMAP(n_neighbors=10, min_dist=0.1, metric="euclidean")
um.fit(X_vis)
X_umap = um.transform(X_vis)

plot_digits_embedding(X_umap, y_vis, 
               "UMAP embedding of the digits (time %.2fs)" %
               (time() - t0))

If time permits, you can try combining the methods above: for instance, use PCA to reduce the dimensionality to accelerate either MDS or tSNE. What do you observe?

In [ ]:
#TODO_OPTIONAL

## The same plots with Bokeh

We are going to improve the visualization of `X_projected`, `X_pca`, `X_mds` and `X_tsne` by using `bokeh`.


If the Bokeh interactive plots don't show as expected, you might have to revert to a previous version of Bokeh (run `!pip install bokeh==1.3.4`) and restart your runtime.

In [ ]:
import bokeh
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
import matplotlib as mpl
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

You are provided with two helper functions to transform images to bokeh-compatible formats. Replace the call to `circle` with `image` and `image_rgba` to change the glyphs.

In [ ]:
def im_colorize_rgba(arr, label):
  color = mpl.cm.tab10(label)
  if arr.ndim == 1:
    out_img = (arr[:, np.newaxis] * np.array(color)).reshape(28,28,4) * 255
  elif arr.ndim == 2:
    out_img = (arr[:, :, np.newaxis] * np.array(color)) * 255
  else:
    raise ValueError("Image format not handled (yet).")
  out_img = np.flipud(out_img)
  return np.squeeze(out_img.astype(np.uint8)).view(np.uint32)

def im_reshape_bw(arr):
  out_img = arr.reshape(28,28)*255
  out_img = np.flipud(out_img)
  return out_img
  
def plot_interactive_embedding(X, y, X_img, title=None):
  bokeh_data = ColumnDataSource(
          data=dict(
              x1 = X[:,0],
              x2 = X[:,1],
              colors = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 
                        255*mpl.cm.tab10(y)],
              images = [im_reshape_bw(xi) for xi in X_img],
              images_rgba = [im_colorize_rgba(xi, yi) for xi, yi in zip(X_img, y)],
              labels = [str(yi) for yi in y]
          )
      )
  hover_tsne = HoverTool(tooltips='@labels')
  tools_tsne = [hover_tsne, 'pan', 'wheel_zoom', 'reset']
  plot_tsne = figure(plot_width=600, plot_height=600, tools=tools_tsne, title=title)

  plot_tsne.circle("x1", "x2", size=4, fill_color="colors", line_width=0, source=bokeh_data)

  return plot_tsne

In [ ]:
fig_tsne = plot_interactive_embedding(X_tsne, y_vis, X_vis, 
               "t-SNE embedding of the digits")

In [ ]:
show(fig_tsne)

In [ ]:
fig_umap = plot_interactive_embedding(X_umap, y_vis, X_vis, 
               "UMAP embedding of the digits")
show(fig_umap)

# 2. Visualizing and debugging neural networks

In the next section, we're going to reproduce what you're learnt before in your machine learning courses with a focus on the visualizing part of it. You should come out with handy functions that you can reuse later for your own experiments.

We will use Fashion-MNIST because that's more fun to visualize.

Build an image classifier & visualize layers

In [ ]:
# adapted from https://github.com/ageron/handson-ml2/blob/master/10_neural_nets_with_keras.ipynb
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
assert tf.__version__ >= "2.0"

from tensorflow import keras

In [ ]:
tf.__version__

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
X_train_full.dtype

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
plot_image(X_train[0], reshape_size=None, cmap="binary", ax=None, title=class_names[y_train[0]]);

In [ ]:
n_rows = 5
n_cols = 20
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        ax = plt.subplot(n_rows, n_cols, index + 1)
        plot_image(X_train[index], ax=ax, title=class_names[y_train[index]]);
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
n_vis_samples = 1000
X_vis = X_test[:n_vis_samples]
y_vis = y_test[:n_vis_samples]
X_in = X_vis.reshape(n_vis_samples, 28*28)

In [ ]:
# Visualize Fashion-MNIST from original input space

## A simple fully connected neural network

In [ ]:
model_fc = keras.models.Sequential()
model_fc.add(keras.layers.Flatten(input_shape=[28, 28]))
model_fc.add(keras.layers.Dense(50, activation="relu", name="fc1"))
model_fc.add(keras.layers.Dense(100, activation="relu", name="fc2"))
model_fc.add(keras.layers.Dense(10, activation="softmax", name="cls_1"))

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
keras.utils.plot_model(model_fc, "my_mnist_model.png", show_shapes=True)

In [ ]:
model_fc.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [ ]:
# before fitting, visualize the layer outputs

# TODO

fig_tsne_fc2_rand = plot_interactive_embedding(X_tsne, y_vis, X_vis, 
               "t-SNE embedding of fashion MNIST from feature space fc2, without training")
show(fig_tsne_fc2_rand)

In [ ]:
# now fit the model
history = model_fc.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

### Looking at the learned weights directly

The first layer's weights is a 784xh matrix, so we can observe a few lines of this matrix as images. Can you see patterns emerge?

In [ ]:
layer_weights = model_fc.get_weights()
# TODO explicit layers dimensions & explain what's in this list

In [ ]:
fig, axes = plt.subplots(4, 4)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = layer_weights[0].min(), layer_weights[0].max()

for coef, ax in zip(layer_weights[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=.5 * vmin,
               vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

### Looking at the latent representations

Another way to look at the intermediate layers is through their effect as feature transforms.

Plot a view of the points as transformed at different levels in the network. What can you observe?

In [ ]:
# after fitting
layer_to_plot = ["fc1", "fc2"]

#TODO
fig_tsne_fc1 = plot_interactive_embedding(X_tsne, y_vis, X_vis, 
               "t-SNE embedding of fashion MNIST from feature space fc1, _after_ training")

# TODO
fig_tsne_fc2 = plot_interactive_embedding(X_tsne, y_vis, X_vis, 
               "t-SNE embedding of fashion MNIST from feature space fc2, _after_ training")


In [ ]:
show(fig_tsne_fc1)

In [ ]:
show(fig_tsne_fc2)

_Answer:_

### Debugging learning

Plot the learning curve from the history output of the `fit` call. What do you observe?

In [ ]:
import pandas as pd
import seaborn as sns

#TODO

_Answer:_

### What do predictions look like?

In this part, we're going to look at the output of the classification model itself.

In [ ]:
# Look at a few random predictions
n_test = 10
X_new = X_test[:n_test]
y_proba = model_fc.predict(X_new)
y_proba.round(2)
y_pred = model_fc.predict_classes(X_new)

In [ ]:
#plot images with true & predicted labels on top
plt.figure(figsize=(20, 3))
#TODO
plt.show()

#### The distribution of predictions

You can use `seaborn`, `bokeh`, or raw `matplotlib` to view these distributions. It might be useful to organise the predictions with dataframes.

1. Which classes are predicted the more, the less?
2. What is the distribution of the predicted probabilities for each class? Do you see any trend? Does it raise questions?

Some tips:
- rotate x-axis labels with `plt.xticks(rotation=75)`
- move legend outside the plot with `plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)`
- change labels on axis ticks with `ax.set_xticklabels(class_names);`
- seaborn has both `displot` and `kdeplot`, matplotlib has `hist`...
- a [ridge-plot](https://seaborn.pydata.org/examples/kde_ridgeplot.html) could look nice!

In [ ]:
# compute test predictions
y_proba_test = model_fc.predict(X_test)
y_pred = y_proba_test.argmax(axis=1)

In [ ]:
ax = plt.subplot(111)
#TODO question 1

In [ ]:
# ex. df = pd.DataFrame(data=y_proba_test, columns=class_names)

#### Which examples are the hardest to classify?

Plot hard example and check whether you'd agree with the model (that they are hard).

In [ ]:
# Find worst mistakes
y_proba_test = model_fc.predict(X_test)
mistakes = np.squeeze(np.argwhere(y_pred != y_test))
worst_mistakes = np.argsort(-y_proba_test[mistakes,:].max(axis=1))[:n_test]

In [ ]:
y_pred[mistakes[:10]], y_test[mistakes[:10]]

In [ ]:
y_pred[worst_mistakes], y_test[worst_mistakes]

In [ ]:
plt.figure(figsize=(20, 3))
#TODO
plt.show()

#### Which classes are the most difficult to classify? The most confused?

This kind of question is best answered by looking at the confusion matrix. If you don't know how to do that, you can look at `sklearn.metrics.confusion_matrix` and `sns.heatmap` for help.

In [ ]:
#TODO

#Visualizing CNN models

In [ ]:
X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train_c = (X_train - X_mean) / X_std
X_valid_c = (X_valid - X_mean) / X_std
X_test_c = (X_test - X_mean) / X_std

X_train_c = X_train_c[..., np.newaxis]
X_valid_c = X_valid_c[..., np.newaxis]
X_test_c = X_test_c[..., np.newaxis]

In [ ]:
from functools import partial
import os

convmodel_file = "convnet.h5"

if os.path.exists(convmodel_file):
  model = keras.models.load_model(convmodel_file)
  trained = True
else:
  DefaultConv2D = partial(keras.layers.Conv2D,
                          kernel_size=3, activation='relu', padding="SAME")

  feature_extractor = keras.models.Sequential([
      DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
      keras.layers.MaxPooling2D(pool_size=2),
      DefaultConv2D(filters=128),
      DefaultConv2D(filters=128),
      keras.layers.MaxPooling2D(pool_size=2),
      DefaultConv2D(filters=256),
      DefaultConv2D(filters=256),
      keras.layers.MaxPooling2D(pool_size=2),
      keras.layers.Flatten(),
  ])
  classifier = keras.models.Sequential([
      keras.layers.Dense(units=128, activation='selu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(units=64, activation='selu'),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(units=10, activation='softmax'),
  ])

  model = keras.models.Sequential([feature_extractor, classifier])
  trained = False


In [ ]:
if not trained:
  model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
  history = model.fit(X_train_c[:10000], y_train[:10000], epochs=10, validation_data=(X_valid_c, y_valid))
  model.save(convmodel_file)
score = model.evaluate(X_test_c, y_test)
X_new = X_test_c[:10] # pretend we have new images
y_pred = model.predict(X_new)


You can redo the same visualization as with the feed-forward model. Instead of looking at weights as images, you can plot the CNN's kernel weights as tiny images.

In [ ]:
# plot history

In [ ]:
# plot tSNE or UMAP

If time permits, you can go through the [DeepDream tutorial](https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/examples/tutorials/deepdream/deepdream.ipynb) and adapt it here to visualise the layers of your network.